## 1) Написать SQL запрос чтобы получить дату 1 и 10 выполненного заказа каждого продавца. Из данных в CSV формате необходимо сделать таблицу в любой из SQL СУБД.



### Подключаемся к csv файлу для формирования SQL запросов через duckdb

In [ ]:
#Подключаем дакдакдб для скл запросов из ксв файла
import pandas as pd, duckdb
import numpy as np

df = pd.read_csv("Данные для задания по SQL.csv", sep=';', encoding='cp1251')
con = duckdb.connect()
con.register('sales', df)

#### В датасете есть записи, где количество заказов отрицательное, рассматриваем как возвраты

In [ ]:
res = con.execute("""
                  
-- В течение одного дня компания может совершить несколько заказов, подготавливаем агрегат с суммой заказов за день
WITH agg_day_sum AS ( 
    SELECT
        CompanyID as company_id,
        FactDate as fact_date,   
        SUM(Qty) AS day_qty  --складываем все заказы за день
    FROM sales
    GROUP BY 1, 2
),
-- считаем накопительную сумму с шагом в день по продавцу
cum_sum AS ( 
    SELECT
        company_id,
        fact_date,
        SUM(day_qty) OVER (PARTITION BY company_id ORDER BY fact_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumsum_qty 

--сортируем по дате количество заказов в окне по компании
    FROM agg_day_sum
)
SELECT                        -- 3) берём самую раннюю дату достижения порогов
    company_id,
    MIN(CASE WHEN cumsum_qty >= 1  THEN fact_date END)  AS firstSale,
    MIN(CASE WHEN cumsum_qty >= 10 THEN fact_date END)  AS tenthSale
FROM cum_sum
GROUP BY company_id
ORDER BY company_id

""").fetchdf()

res

## 2) Рассчитать с какой даты должно начаться взимание платы за услугу по продавцам.

Продавцы маркетплейса Y получают бесплатную услугу первые 10 месяцев пользования этой услуги, но они периодически эту услугу могут отключать и подключать заново.
Оплата за услугу начинается взиматься только после 10 полных месяцев пользования услугой. 
Датой началом пользования услуги необходимо считать первое событие подключения услуги.

Задача:
Рассчитать с какой даты должно начаться взимание платы за услугу по продавцам.


In [4]:
import pandas as pd
import numpy as np
df = pd.read_pickle("данные для Python.pickle")  # compression будет определён автоматически
df

,company_id,value,created_at
0,1,0,2021-04-12 11:31:58.598823
1,1,1,2022-01-18 15:20:08.607000
2,2,1,2021-10-01 10:11:44.721200
3,2,1,2021-10-01 10:14:23.059372
4,2,1,2021-09-30 17:00:38.684091
...,...,...,...
42825,41979,1,2022-07-19 07:52:10.433332
42826,41980,1,2022-07-19 10:39:42.332804
42827,41981,1,2022-07-19 07:15:16.024747
42828,41982,1,2022-07-19 09:18:17.709383



1.ищем первое включение value = 1
2.затем проходим по событиям и сверяемся с месячным шагом в 1 месяц
3.как только наберём 10 полный шагов внутри активных интервалов, то возвращаем дату этого 10-го шагов

In [5]:
from dateutil.relativedelta import relativedelta


def calc(group: pd.DataFrame) -> pd.Timestamp:

    # сортируем и превращаем в кортеж по которому можно итерироваться
    events = group.sort_values("created_at")[["value", "created_at"]].to_records(index=False)

# получаем первое включение услуги у компании first_on
    first_on = None
    for value, timestep in events:
        if value == 1:
            first_on = timestep
            break
        # если компания так и не включила вернем nat
    if first_on is None:
        return pd.NaT 

# Для каждой компании пройдемся по интервалам работы и будем включать или выключать статус active в зависимости от value

    active = False
    intervals = []
    cur_time = None

    for val, ts in events:
        if ts < first_on:
            # события до первого включения игнорируем
            continue

        if cur_time is None:
            cur_time = first_on
            active = True  

        if ts == first_on:
            # это по факту первое включение 
            continue

# В список интервалов записывается start, end, is_active начиная с first_on
        intervals.append((cur_time, ts, active))
        # переключаем статус по событию
        active = (val == 1)
        cur_time = ts
    
    first_on = pd.Timestamp(first_on)   # привели к Timestamp

    # Если у компании последняя запись это value = 1, то прибавляем к ней 10+ месяцев, чтобы получить дату закрытия
    period_end = first_on + np.timedelta64(330, 'D')
    intervals.append((cur_time, period_end, active))

    # Теперь итеративно идем по интервалам и от первого включения считаем сколько полных месяцев
    steps_needed = 10
    steps_done = 0
    k = 1
    # пройдём по активным интервалам и засчитаем все тики, которые туда попадают
    for start, end, is_active in intervals:
        start = pd.Timestamp(start)
        end = pd.Timestamp(end)

        if not is_active:
            continue
        # print(f"start {start}, end {end}, is_active {is_active}")
        # берём последовательность контрольных дат first_on + k месяцев, которые >= start и < end
        while steps_done < steps_needed:
            step = first_on + relativedelta(months=k)
            # print(f"step {step}")

            if step >= end:
                break
            if step >= start:
                steps_done += 1
                if steps_done == steps_needed:
                    return step
            k += 1

        if steps_done == steps_needed:
            break

        if steps_done < steps_needed:
            while first_on + relativedelta(months=k) < start:
                k += 1
        # print(step, steps_done)

    return pd.NaT

# группируем по компаниям и каждой для каждой компании считаем дату оплаты подписки
dates = (
    df.groupby("company_id", as_index=False)
      .apply(calc)
      .rename(columns={None: "billing_start"})
)


C:\Users\shela\AppData\Local\Temp\ipykernel_27492\3713100080.py:88: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calc)


In [ ]:
dates.to_pickle("billing_start.pkl", protocol=4)